In [1]:
import pandas as pd

In [3]:
test = pd.read_csv('/Users/blytheluo/Desktop/test.csv')
train = pd.read_csv('/Users/blytheluo/Desktop/train.csv')
video_info = pd.read_csv('/Users/blytheluo/Desktop/video_info.csv')

In [4]:
train.head()

,userId,VideoId,label
0,9118,2235,3
1,5223,3131,5
2,807,2730,4
3,11871,1817,5
4,6831,390,4


In [5]:
test.head()

,number,userId,VideoId
0,0,13496,2525
1,1,2117,437
2,2,19472,1216
3,3,18838,914
4,4,8138,2597


In [8]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 28 kB/s eta 0:00:01
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp38-cp38-macosx_10_9_x86_64.whl size=764475 sha256=b77835eea17d167ab5632f5db6f95df08742e4d73a45c4dfda417ed2f28acf98
  Stored in directory: /Users/blytheluo/Library/Caches/pip/wheels/20/91/57/2965d4cff1b8ac7ed1b6fa25741882af3974b54a31759e10b6
Successfully built scikit-surprise


In [9]:
from surprise import Reader, Dataset, SVD, NMF
from surprise.model_selection import cross_validate
from surprise import dataset

In [10]:
video_info.head()

,VideoId,keyWords,type,length,heat,rating
0,761,"5,37,8,47,1,61,6",6.0,13,143.53,4.630
1,2338,"5,64,65,51,48,73,6",6.0,11,123.19,4.625
2,820,"5,64,65,51,48,73,6",6.0,11,126.14,4.580
3,1930,"19,8,62,6,9",6.0,3,121.07,4.575
4,3519,"5,64,65,51,48,73,6",6.0,3,119.60,4.555


In [11]:
reader = Reader(line_format='user item rating', rating_scale=(0,5))

In [15]:
class MyDataset(dataset.DatasetAutoFolds):
    def __init__(self, df, reader):
        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in 
                            zip(df['userID'], df['VideoID'], df['label'])]
        self.reader=reader
data = MyDataset(train[['userID','VideoID', 'label']], reader)

KeyError: "['userID', 'VideoID'] not in index"

In [17]:
class MyDataset(dataset.DatasetAutoFolds):
    def __init__(self, df, reader):
        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(df['userId'], df['VideoId'], df['label'])]
        self.reader=reader
data = MyDataset(train[['userId', 'VideoId','label']], reader)

In [19]:
pre=[]
valid=[]
from surprise.model_selection import KFold
from surprise import accuracy

kf = KFold(n_splits=5)
for trainset, testset in kf.split(data):
    algo = NMF()
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)
    for prediction in predictions:
          valid.append([prediction.uid,prediction.iid,prediction.est])
    for i in range(len(test)):
        uu=test.loc[i,'userId']
        ii=test.loc[i,'VideoId']
        es=algo.predict(uu,ii).est
        pre.append([i,uu,ii,es])

RMSE: 0.6717
RMSE: 0.6715
RMSE: 0.6707
RMSE: 0.6688
RMSE: 0.6705


In [20]:
len(pre)

730835

In [21]:
prediction=[i[3] for i in pre]

In [22]:
test['pre1']=prediction[:1270378]
test['pre2']=prediction[1270378:1270378*2]
test['pre3']=prediction[1270378*2:1270378*3]
test['pre4']=prediction[1270378*3:1270378*4]
test['pre5']=prediction[1270378*4:1270378*5]

ValueError: Length of values (730835) does not match length of index (146167)

In [23]:
test['pre']=(test['pre1']+test['pre2']+test['pre3']+test['pre4']+test['pre5'])/5

KeyError: 'pre1'

In [24]:
def score(y,y_pred):
    return 1-sum((y-y_pred)**2)/len(y)

In [26]:
print(score(test['number'],test['pre']))

KeyError: 'pre'